In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install bokeh

In [5]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

Loading BokehJS ...

In [6]:
dataset = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv")
dataset.head()

,PPSORT,WEIGHT,WT1,WT2,WT3,WT4,WT5,WT6,WT7,WT8,...,Subsidy,Tenur,TotInc,TotInc_AT,VALUE,VisMin,Wages,WKSWRK,WRKACT,YRIMM
0,453141,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,97000,73000,450000,13,95000,6,11,9999
1,923226,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,99999999,99999999,440000,13,99999999,9,99,9999
2,385097,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,99999999,99999999,440000,13,99999999,9,99,9999
3,732612,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,592.596438,...,9,1,46000,41000,839779,13,19000,6,11,9999
4,143665,37.120914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,30000,26000,60000,13,29000,5,9,9999


In [12]:
list(dataset.columns)

['PPSORT',
 'WEIGHT',
 'WT1',
 'WT2',
 'WT3',
 'WT4',
 'WT5',
 'WT6',
 'WT7',
 'WT8',
 'WT9',
 'WT10',
 'WT11',
 'WT12',
 'WT13',
 'WT14',
 'WT15',
 'WT16',
 'ABOID',
 'AGEGRP',
 'AGEIMM',
 'ATTSCH',
 'BedRm',
 'BFNMEMB',
 'CapGn',
 'CFInc',
 'CFInc_AT',
 'CfSize',
 'CFSTAT',
 'CHDBN',
 'ChldC',
 'CIP2011',
 'CIP2011_STEM_SUM',
 'Citizen',
 'CitOth',
 'CMA',
 'CONDO',
 'COW',
 'CQPPB',
 'DETH123',
 'DIST',
 'DPGRSUM',
 'DTYPE',
 'EFDecile',
 'EfDIMBM',
 'EFInc',
 'EFInc_AT',
 'EfSize',
 'EICBN',
 'EmpIn',
 'ETHDER',
 'FOL',
 'FPTWK',
 'GENSTAT',
 'GovtI',
 'GTRfs',
 'HCORENEED_IND',
 'HDGREE',
 'HHInc',
 'HHInc_AT',
 'HHMRKINC',
 'HHSIZE',
 'HHTYPE',
 'HLAEN',
 'HLAFR',
 'HLANO',
 'HLBEN',
 'HLBFR',
 'HLBNO',
 'IMMCAT5',
 'IMMSTAT',
 'IncTax',
 'Invst',
 'KOL',
 'LFACT',
 'LICO',
 'LICO_AT',
 'LOC_ST_RES',
 'LOCSTUD',
 'LoLIMA',
 'LoLIMB',
 'LoMBM',
 'LSTWRK',
 'LWAEN',
 'LWAFR',
 'LWANO',
 'LWBEN',
 'LWBFR',
 'LWBNO',
 'MarStH',
 'MOB1',
 'Mob5',
 'MODE',
 'MrkInc',
 'MTNEn',
 'MTNFr'

In [17]:
#removing English as Mother Tongue before proceeding:
datasetNoMTNEN = dataset.loc[dataset['MTNEn'] != 1]
print("Number of samples when English removed as mother tongue:", datasetNoMTNEN.shape[0])
datasetNoMTNEN = datasetNoMTNEN.loc[datasetNoMTNEN['MTNEn'] != 8]
print("Number of samples when unavailable rows also removed:", datasetNoMTNEN.shape[0])

Number of samples when English removed as mother tongue: 389571
Number of samples when unavailable rows also removed: 387817


In [28]:
datasetNoMTNEN.shape

(387817, 141)

###  Notes from Mr. Fogel

MTNNO (Mother Tongue - first write-in component) and HLANO (Home Language - first write-in component) - see if these match, use boolean whether they match as classification. Also classify MTNNO and HLANO separately

In Decision Tree file I pruned out these values from feature set - these all come from the language section:
- MTNEN (mother tongue english) - high correlation to classified feature in tree
- MTNFR (mother tongue french)   - ditto
- HLAFR (home language french)  - ditto
- HLAEN (home language english) - ditto
- HLBEN (home language part b english)
- HLBFR (home language part b french) 
- HLBNO 
- NOL (knowledge of non-official languages) - very high correlation to classified features
- FOL (first official language spoken) 
- KOL (knowledge of official languages) 

_----the below language features had very low influence in the decision tree----_

- LWAEN (language at work part a english)
- LWAFR (language at work part a french)
- LWANO (language at work part a first write-in component)
- LWBEN (language at work part b english)
- LWBFR (language at work part b french)
- LWBNO (language at work part b first write-in component)

In [22]:
# class label: home language part A - first language write in component
homeLang = datasetNoMTNEN.iloc[:,65]
# class label: mother tongue part A - first language write in component
motherTongue = datasetNoMTNEN.iloc[:, 96]
# variables
x1, x2, x3 = datasetNoMTNEN.iloc[:, 18:65], datasetNoMTNEN.iloc[:, 66:96], datasetNoMTNEN.iloc[:, 97:-1]
x = pd.concat([x1,x2,x3], axis=1, sort=False)
weights = datasetNoMTNEN.iloc[:, 1]

In [27]:
x.tail()

,ABOID,AGEGRP,AGEIMM,ATTSCH,BedRm,BFNMEMB,CapGn,CFInc,CFInc_AT,CfSize,...,SSGRAD,Subsidy,Tenur,TotInc,TotInc_AT,VALUE,VisMin,Wages,WKSWRK,WRKACT
930407,6,14,8,1,3,0,99999999,25,21,3,...,8,9,1,64000,49000,390000,4,64000,6,11
930410,6,11,99,1,4,0,99999999,30,25,4,...,5,9,1,26000,24000,230000,13,22000,5,10
930411,6,11,99,1,4,0,99999999,30,25,4,...,5,9,1,130000,83000,240000,13,120000,3,5
930412,6,3,99,9,4,0,99999999,30,25,4,...,99,9,1,99999999,99999999,250000,13,99999999,9,99
930419,6,21,7,1,3,0,99999999,13,13,3,...,1,0,2,22000,23000,99999999,2,99999999,9,1


When English as a mother tongue is present 88% of the entries for age of immigration are unavailable/unapplicable. 

Once English as a mother tongue has been removed this goes down to 61%. 

In [26]:
datasetNoMTNEN[(datasetNoMTNEN['AGEIMM'] == 88) | (datasetNoMTNEN['AGEIMM'] == 99)].shape[0]/datasetNoMTNEN.shape[0]

0.6161230683544042

CitOth : Citizenship: Other country of citizenship has the value 99 
        to indicate no other country of citizenship, while 88 is 'not available'

In [41]:
missing_values_dict = dict(
    AGEGRP=88,
    MOB1= [8,9],
    Mob5=9,
    PR1=[88,99],
    PR5=99,
    DPGRSUM=88,
    ETHDER=88,
    VisMin=88,
    HLANO=88, # This is one of the target columns
    KOL=8,
    LWAEN=[8,9],
    LWAFR=[8,9],
    LWANO=9,
    LWBEN=[8,9],
    LWBFR=[8,9],
    LWBNO=9,
    MTNEn=8,
    MTNFr=8,
    MTNNO=88, # This is one of the target columns
    NOL=88,
    AGEIMM=[88,99],
    CitOth=88,
    GENSTAT=8,
    IMMCAT5=88,
    IMMSTAT=8,
    POB=88,
    POBF=8,
    POBM=8,
    YRIMM=[8888,9999],
    ATTSCH=[8,9],
    CIP2011=[88,99],
    CIP2011_STEM_SUM=[88,99],
    HDGREE=[88,99],
    LOC_ST_RES=9,
    LOCSTUD=99,
    SSGRAD=[88,99],
    COW=8,
    FPTWK=[8,9],
    LFACT=99,
    LSTWRK=9,
    NAICS=[88,99],
    NOC16=[88,99],
    NOCS=[88,99],
    WKSWRK=9,
    WRKACT=[88,99],
    DIST=[8,9],
    MODE=9,
    POWST=[8,9],
    PWDUR=[8,9],
    PWLEAVE=[8,9],
    PWOCC=9,
    PWPR=[88,99],
    CapGn=[88888888,99999999],
    CFInc=88,
    CFInc_AT=88,
    CHDBN=[88888888,99999999],
    ChldC=[88888888,99999999],
    CQPPB=[88888888,99999999],
    EFDecile=88,
    EfDIMBM=88,
    EFInc=88,
    EFInc_AT=88,
    EICBN=[88888888,99999999],
    EmpIn=[88888888,99999999],
    GovtI=[88888888,99999999],
    GTRfs=[88888888,99999999],
    HHInc=88,
    HHInc_AT=88,
    HHMRKINC=88,
    IncTax=[88888888,99999999],
    Invst=[88888888,99999999],
    LICO=[8,9],
    LICO_AT=[8,9],
    LoLIMA=[8,9],
    LoLIMB=[8,9],
    LoMBM=[8,9],
    MrkInc=[88888888,99999999],
    OASGI=[88888888,99999999],
    OtInc=[88888888,99999999],
    Retir=[88888888,99999999],
    SempI=[88888888,99999999],
    TotInc=[88888888,99999999],
    TotInc_AT=[88888888,99999999],
    Wages=[88888888,99999999],
    CfSize=8,
    EfSize=8,
    PKID0_1=[8,9],
    PKID15_24=[8,9],
    PKID2_5=[8,9],
    PKID25=[8,9],
    PKID6_14=[8,9],
    PKIDS=9,
    HHSIZE=8,
    BedRm=8,
    CONDO=8,
    DTYPE=8,
    HCORENEED_IND=888,
    NOS=8,
    PresMortG=[8,9],
    REPAIR=8,
    ROOMS=88,
    Subsidy=[8,9],
    Tenur=8,
    VALUE=[88888888,99999999]
)

In [67]:

def process_NA_rows(dataset, col_name, na_value):
    if isinstance(na_value, list):
        num_na = dataset.loc[dataset[col_name] == na_value[0]].shape[0]
        num_na += dataset.loc[dataset[col_name] == na_value[1]].shape[0]
    else:
        num_na = dataset[dataset[col_name] == na_value].shape[0]
    print(col_name, "% missing values =", num_na/dataset.shape[0]*100)
    ratio = num_na/dataset.shape[0]*100
    return ratio  

In [68]:
missing_val_summary = dict()
for name in list(datasetNoMTNEN.columns):
    if name in missing_values_dict:
        percent_bad_rows = process_NA_rows(datasetNoMTNEN, name, missing_values_dict[name])
        missing_val_summary[name] = percent_bad_rows
    

AGEGRP % missing values = 1.1484798242470031
AGEIMM % missing values = 61.61230683544042
ATTSCH % missing values = 14.244089351420902
BedRm % missing values = 0.9744286609405983
CapGn % missing values = 91.65224835424955
CFInc % missing values = 0.7410711753223814
CFInc_AT % missing values = 0.7410711753223814
CfSize % missing values = 0.3537751052687221
CHDBN % missing values = 87.80017379330974
ChldC % missing values = 94.32206427258217
CIP2011 % missing values = 15.771356077737696
CIP2011_STEM_SUM % missing values = 15.770582517011889
CitOth % missing values = 2.3057266700531436
CONDO % missing values = 0.6345776487363886
COW % missing values = 0.6278734557794011
CQPPB % missing values = 79.99700889852687
DIST % missing values = 53.99170227194785
DPGRSUM % missing values = 2.9642847012895257
DTYPE % missing values = 0.29292166150529764
EFDecile % missing values = 0.7410711753223814
EfDIMBM % missing values = 0.7410711753223814
EFInc % missing values = 0.7410711753223814
EFInc_AT % m

In [69]:


df_missing = pd.DataFrame(list(missing_val_summary.items()), columns=['Feature', 'Ratio'])
df_missing

,Feature,Ratio
0,AGEGRP,1.148480
1,AGEIMM,61.612307
2,ATTSCH,14.244089
3,BedRm,0.974429
4,CapGn,91.652248
5,CFInc,0.741071
6,CFInc_AT,0.741071
7,CfSize,0.353775
8,CHDBN,87.800174
9,ChldC,94.322064


In [70]:
df_missing = df_missing.sort_values(by=['Ratio'], ascending=False)
source = ColumnDataSource(data=df_missing)

p = figure(x_range=df_missing['Feature'], plot_height=500, plot_width=2000, title="Missing Ratios",
           toolbar_location=None, x_axis_label='Feature', y_axis_label='Ratio', tools="")
p.vbar(x='Feature', top='Ratio', width=0.9, source=source)

p.xaxis.major_label_orientation = np.pi/4
p.xgrid.grid_line_color = None
p.y_range.start = 0

p.add_tools(HoverTool(tooltips=[('Feature', '@Feature'),
                                ('Ratio', '@Ratio')]))

show(p)

Looking at the above data, a logical point to prune features would be those that are over 30% missing ratio. Proceeding with this option

In [13]:
df_missing_pruned = df_missing[df_missing['Ratio'] > 0.3]
df_missing_pruned

,Feature,Count,Ratio
56,ChldC,886332,0.952614
62,EICBN,862277,0.926760
80,SempI,849594,0.913129
52,CapGn,844416,0.907563
55,CHDBN,828655,0.890624
79,Retir,817735,0.878887
21,CitOth,817447,0.878578
78,OtInc,810831,0.871467
77,OASGI,795379,0.854859
57,CQPPB,752220,0.808473


In [14]:
len(df_missing_pruned)

39

Feature set needs to be further pruned...

In [15]:
df_unpruned = df_missing[df_missing['Ratio'] <= 0.3]
to_impute = df_unpruned['Feature'].tolist()
to_impute

['MrkInc',
 'VALUE',
 'PresMortG',
 'TotInc',
 'TotInc_AT',
 'PR1',
 'PKID0_1',
 'PKID2_5',
 'PKID25',
 'PKID6_14',
 'PKIDS',
 'PKID15_24',
 'CIP2011',
 'CIP2011_STEM_SUM',
 'LFACT',
 'HDGREE',
 'LOC_ST_RES',
 'WRKACT',
 'SSGRAD',
 'LSTWRK',
 'ATTSCH',
 'PR5',
 'Mob5',
 'HCORENEED_IND',
 'POBF',
 'POBM',
 'ETHDER',
 'DPGRSUM',
 'VisMin',
 'POB',
 'NOL',
 'MTNNO',
 'MOB1',
 'AGEGRP',
 'BedRm',
 'ROOMS',
 'NOS',
 'HHInc_AT',
 'EFDecile',
 'EfDIMBM',
 'EFInc_AT',
 'EFInc',
 'HHMRKINC',
 'CFInc_AT',
 'CFInc',
 'HHInc',
 'CONDO',
 'Tenur',
 'HLANO',
 'HHSIZE',
 'CfSize',
 'EfSize',
 'DTYPE',
 'LoLIMA',
 'LoLIMB',
 'LICO',
 'LoMBM',
 'LICO_AT',
 'IMMCAT5',
 'IMMSTAT',
 'MTNFr',
 'MTNEn',
 'KOL',
 'REPAIR',
 'GENSTAT']

In [16]:
df_unpruned.shape[0]

65

####  Data Imputation

Need to impute the dummy values used in the data, e.g. the “99999999” seen in the Wages column. The PDF metadata tells you for each column what it used for its dummy values.

Simplest solution: Remove the rows which have these values.

Simple solution: take the average (mean/median/mode for numerical, median/mode for categorical) of the non-dummy values and replace dummy values by the average found.

Ultra fun solution: train a model (like our best friend, XGBoost) to predict the missing values.


#### Going with approach 2: replace dummy values with mean for continuous variables and mode for categorical variables

In [17]:
imputed = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", usecols=to_impute)

In [18]:
na_dict = {
     'MrkInc':[88888888, 99999999],
     'VALUE':[88888888, 99999999],
     'PresMortG':[9],
     'TotInc':[88888888, 99999999],
     'TotInc_AT':[88888888, 99999999],
     'PR1':[99],
     'PKID0_1':[9],
     'PKID15_24':[9],
     'PKID2_5':[9],
     'PKID25':[9],
     'PKIDS':[9],
     'PKID6_14':[9],
     'WRKACT':[99],
     'LOC_ST_RES':[9],
     'HDGREE':[99],
     'CIP2011_STEM_SUM':[99],
     'SSGRAD':[99],
     'CIP2011':[99],
     'LFACT':[99],
     'LSTWRK':[9],
     'ATTSCH':[9],
     'PR5':[99],
     'Mob5':[9],
     'HCORENEED_IND':[888],
     'POBF':[8],
     'POBM':[8],
     'ETHDER':[88],
     'DPGRSUM':[88],
     'VisMin':[88],
     'POB':[88],
     'NOL':[88],
     'MTNNO':[88], # This is one of the target columns
     'MOB1':[9],
     'AGEGRP':[88],
     'BedRm':[8],
     'ROOMS':[88],
     'NOS':[8],
     'EFInc_AT':[88],
     'CFInc_AT':[88],
     'EFDecile':[88],
     'EfDIMBM':[88],
     'CFInc':[88],
     'HHInc':[88],
     'HHInc_AT':[88],
     'HHMRKINC':[88],
     'EFInc':[88],
     'CONDO':[8],
     'Tenur':[8],
     'HLANO':[9], # This is one of the target columns
     'HHSIZE':[8],
     'CfSize':[8],
     'EfSize':[8],
     'DTYPE':[8],
     'LoLIMB':[9],
     'LoMBM':[9],
     'LICO':[9],
     'LICO_AT':[9],
     'LoLIMA':[9],
     'IMMCAT5':[88],
     'IMMSTAT':[8],
     'MTNFr':[8],
     'MTNEn':[8],
     'KOL':[8],
     'REPAIR':[8],
     'GENSTAT':[8]
}

In [19]:
imputed['MrkInc'].value_counts()

 99999999    267117
 1000         19921
 2000         13895
 3000         12954
 1            12610
              ...  
-41000            2
-48000            2
 500000           1
-39000            1
-46000            1
Name: MrkInc, Length: 265, dtype: int64

In [20]:
# Replacing the dummy values with the median for each of the columns
# for col in na_dict.keys():
#     curr = imputed[col]
#     valid = curr
#     for val in na_dict[col]:
#         valid = valid[valid != val]
#     median = valid.median()
# #     print(f'median for {col} is {median}')
#     for val in na_dict[col]:
#         curr[curr == val] = median

In [21]:
# Replacing the dummy values
for col in na_dict.keys():
    curr = imputed[col]
    valid = curr
    vals = na_dict[col]
    if len(vals) > 1: # variable is continuous
        for val in vals:
            valid = valid[valid != val]
        average = int(valid.mean())
    else: # variable is categorical
        valid = valid[valid != vals[0]]
        average = valid.mode().iloc[0]
#     print(f'average for {col} is {average}')
    for val in vals:
        curr[curr == val] = average

In [22]:
imputed['MrkInc'].value_counts()

 49131     272659
 1000       19921
 2000       13895
 3000       12954
 1          12610
            ...  
-47000          2
-41000          2
-39000          1
-46000          1
 500000         1
Name: MrkInc, Length: 264, dtype: int64

### Now that the selected features have been imputed, combine with the original dataset to have preprocessed data for model fitting

In [23]:
all_cols = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", nrows=1).columns
unimputed_cols = list(set(all_cols) - set(missing_dict.keys()))
df_selected = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", usecols=unimputed_cols)
df_selected = pd.concat([df_selected, imputed], axis=1, sort=False).reset_index()

df_selected

,index,PPSORT,WEIGHT,WT1,WT2,WT3,WT4,WT5,WT6,WT7,...,PresMortG,REPAIR,ROOMS,SSGRAD,Tenur,TotInc,TotInc_AT,VALUE,VisMin,WRKACT
0,0,453141,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,5,1,97000,73000,450000,13,11
1,1,923226,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,4,1,49071,39759,440000,13,11
2,2,385097,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,4,1,49071,39759,440000,13,11
3,3,732612,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,2,8,8,1,46000,41000,839779,13,11
4,4,143665,37.120914,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,3,5,6,1,30000,26000,60000,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930416,930416,700854,37.037277,0.0,0.0,0.000000,0.000000,0.0,592.596438,0.0,...,0,1,11,8,1,120000,94000,810000,13,11
930417,930417,821443,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,8,1,5,2,1,77000,61000,494190,13,11
930418,930418,116531,37.042280,0.0,0.0,592.676474,0.000000,0.0,0.000000,0.0,...,1,2,5,6,1,80000,62000,710000,13,9
930419,930419,499993,37.037277,0.0,0.0,0.000000,592.596438,0.0,0.000000,0.0,...,1,2,7,1,2,22000,23000,494190,2,1


### Writing data with pruned and imputed columns to new csv

In [24]:
df_selected.to_csv('pumf-2016-selected-features.csv', index=False)

In [26]:
df_selected.columns.values

array(['index', 'PPSORT', 'WEIGHT', 'WT1', 'WT2', 'WT3', 'WT4', 'WT5',
       'WT6', 'WT7', 'WT8', 'WT9', 'WT10', 'WT11', 'WT12', 'WT13', 'WT14',
       'WT15', 'WT16', 'ABOID', 'BFNMEMB', 'CFSTAT', 'Citizen', 'CMA',
       'DETH123', 'FOL', 'HHTYPE', 'HLAEN', 'HLAFR', 'HLBEN', 'HLBFR',
       'HLBNO', 'MarStH', 'PR', 'PRIHM', 'REGIND', 'Sex', 'SHELCO',
       'AGEGRP', 'ATTSCH', 'BedRm', 'CFInc', 'CFInc_AT', 'CfSize',
       'CIP2011', 'CIP2011_STEM_SUM', 'CONDO', 'DPGRSUM', 'DTYPE',
       'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize', 'ETHDER',
       'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc', 'HHInc_AT',
       'HHMRKINC', 'HHSIZE', 'HLANO', 'IMMCAT5', 'IMMSTAT', 'KOL',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MOB1', 'Mob5', 'MrkInc', 'MTNEn', 'MTNFr',
       'MTNNO', 'NOL', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5', 'PKID25',
       'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR1', 'PR5',
       'PresMortG', 'REPAIR